# Simple FRAGMENT-MNP demo

The simplest fragmentation model is one that presumes that the only loss/gain from individual size classes is from fragmentation, neglecting other processes such as aggregation and dissolution. This can be represented by:

$$
\frac{dn_k}{dt} = -k_{\text{frag},k} n_k + \Sigma_i f_{i,k} k_{\text{frag},i} n_i
$$

where $n_k$ is the particle number concentration in size class $k$, $t$ is time, $k_{\text{frag},k}$ is the fragmentation rate of size class $k$, and $f_{i,k}$ is the fraction of daughter fragments produced from a fragmenting particle of size $i$ that are of size $k$. Note that there is no source term - the only input of plastic particles is the initial value at $t=0$. We can solve this numerically, which the following does using the specified parameters.

In [3]:
from scipy.integrate import solve_ivp
import numpy as np
from ipywidgets import widgets, interact
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib import cm
from matplotlib.colors import rgb2hex

style = {'description_width': '250px'}
layout = widgets.Layout(width='600px')

@interact
def fragmentmnp(K=widgets.IntSlider(min=1, max=100, value=7, description="Number of size classes", style=style, layout=layout),
                K_range=widgets.IntRangeSlider(min=-15, max=-2, value=[-9, -3], description="Size class diameter range (10^value)", style=style, layout=layout),
                T=widgets.IntSlider(min=1, max=1000, value=100, description="Number of timesteps", style=style, layout=layout),
                bar_ts=widgets.IntText(value=99, description="Show bar chart for time step...", style=style, layout=layout),
                _k_frag=widgets.FloatText(0.01, description="Fragmentation rate", style=style, layout=layout),
                _n_0=widgets.FloatText(42.0, description="Initial particle number concentration", style=style, layout=layout),
                loss=widgets.Checkbox(value=True, description="Allow fragmentation from smallest size class?", style=style, layout=layout)):

  # Set vectors across the size classes based on constant inputs
  k_frag = np.full(K, _k_frag)
  # If we fragmentation from smallest class isn't allowed, then set k_frag for
  # that class to zero
  if not loss:
    k_frag[K-1] = 0.0
  n_0 = np.full(K, _n_0)

  # Fragment size distribution matrix - assume fragmentation event results in even
  # split between size classes of daughter particles
  fsd = np.zeros((K,K))
  for k in np.arange(K):
    fsd[k,:] = 1 / (K - k - 1) if (K - k) != 1 else 0
    # Get the upper triangle of this matrix, which effectively sets f to zero for
    # size classes larger (or equal to) than the current one
  fsd = np.triu(fsd, k=1)

  # Define the function that satisfies n'(t) = f(t, n)
  # i.e. the RHS of our differential eq
  def f(t, n):
    # Get number of size classes and create empty result to be filled
    N = n.shape[0]
    dndt = np.empty(N)
    # Loop over the size classes and perform the calculation
    for k in np.arange(N):
      dndt[k] = - k_frag[k] * n[k] + np.sum(fsd[:,k] * k_frag * n)
    # Return the solution for all of the size classes
    return dndt

  # Numerically solve this given the initial values for n, over T time steps
  soln = solve_ivp(fun=f,
                   t_span=(0, T),
                   y0=n_0,
                   t_eval=np.arange(0, T))

  # If k_frag != 0 for the smallest size class, then there will be a loss to the
  # system, so keep track of that here
  n_loss = np.sum(n_0) - np.sum(soln.y, axis=0)

  # Set up a particle size distribution - only needed to print legend
  d = np.logspace(K_range[1], K_range[0], K)

  # Finally, create the graphs!

  # Define the colour map to use
  viridis = [rgb2hex(rgb) for rgb in cm.get_cmap('viridis', K).colors]
  
  # Set up the subplots
  fig = make_subplots(rows=1, cols=2, shared_yaxes=True, horizontal_spacing=0.03,
                      subplot_titles=('', f'Time step: {bar_ts}'))

  # Line chart timeseries first
  for i in range(0,K):
    fig.add_trace(go.Scatter(x=soln.t, y=soln.y[i], name=f'{d[i]:.2e} m',
                  line_color=viridis[i]), row=1, col=1)
  if loss:
    # Plot the loss (if there is any) with a different style
    fig.add_trace(go.Scatter(x=soln.t, y=n_loss, name='Loss',
                             line={'width': 3, 'dash': 'dash', 'color': 'lightblue'}), row=1, col=1)
  fig.update_xaxes(title='Time', col=1, row=1)
  fig.update_yaxes(title='Particle number concentration', col=1, row=1)
  # Add the time slice line
  # fig.add_shape(type='line', x0=bar_ts, y0=0, x1=bar_ts, y1=soln.y[K-1].max(),
  #               line={'color': 'red', 'width': 3})
  
  # Now the bar chart
  if bar_ts < T:
    fig.append_trace(go.Bar(x=[f'{sc:.2e} m' for sc in d], y=soln.y[:,bar_ts],
                            marker_color=viridis, showlegend=False), row=1, col=2)
  fig.update_xaxes(title='Size class diameter', col=2, row=1)

  # Update the layout and show
  fig.update_layout(width=1300)
  fig.show()

interactive(children=(IntSlider(value=7, description='Number of size classes', layout=Layout(width='600px'), m…